In [2]:
from ultralytics import YOLO
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO('yolov8s.yaml').load('yolov8s.pt')
model_path = 'models/yolov8/detect/train/weights/best.pt'
model = YOLO(model_path).to(device)
print(device)

Transferred 355/355 items from pretrained weights
cuda


In [8]:
import cv2
import json

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  

    if not ret:
        break

    res = model(frame, verbose=False)
    predictions = json.loads(res[0].tojson())
    for pred in predictions:
        conf_score = pred['confidence']

        if(conf_score < 0.7):
            continue

        bbox = pred['box']
        x1, y1 = int(bbox['x1']), int(bbox['y1'])
        x2, y2 = int(bbox['x2']), int(bbox['y2'])

        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        frame = cv2.putText(frame, "", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
